# Profiler

Напишите декоратор `@profiler`, который при вызове функции будет замерять время ее исполнения

Для работы со временем в питоне есть замечательный модуль `datetime`.

Декоратор не должен затирать основные атрибуты функции: `__name__`, `__doc__`, `__module__`. Вам понадобится одна строчка дополнительная строчка для этого (см. ноутбук по теме)

Пользоваться глобальными переменными запрещено, сохранять результаты замера нужно в **атрибуте** функции.
Атрибут назовите `last_time_taken`.


> Вообще, хранить какие-то свои данные в атрибутах функции - антипаттерн, и в продакшен коде так делать не стоит.


In [7]:
from functools import wraps
from datetime import datetime, timedelta

In [8]:
def profiler(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.now()
        value = func(*args, **kwargs)
        wrapper.last_time_taken = datetime.now() - start
        return value
    
    wrapper.last_time_taken = timedelta(0)

    return wrapper


In [9]:
@profiler
def foo():
    pass

foo()


assert foo.last_time_taken > timedelta(0)
print(f'Time: {foo.last_time_taken}')

Time: 0:00:00.000006


# Calls counter

Напишите декоратор `@calls_counter`, который при вызове функции будет замерять количество рекусивных вызовов

Декоратор не должен затирать основные атрибуты функции: `__name__`, `__doc__`, `__module__`. Вам понадобится одна строчка дополнительная строчка для этого (см. ноутбук по теме)

Пользоваться глобальными переменными запрещено, сохранять результаты замера нужно в **атрибуте** функции.
Атрибут назовите `calls`.

In [10]:
# если под количеством рекурсивных вызовов
# подразумевалось количество вызовов функции
# def calls_counter(func):

#     def wrapper(*args, **kwargs):
#         wrapper.calls += 1
#         return func(*args, **kwargs)

#     wrapper.calls = 0
#     return wrapper


# если под количеством рекурсивных вызовов
# понималась глубина рекурсии последнего вызова
# def calls_counter(func):
#     depth = 0

#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         nonlocal depth
#         if depth == 0:
#             wrapper.calls = 0
#         depth += 1
#         wrapper.calls = max(wrapper.calls, depth)
#         value =  func(*args, **kwargs)
#         depth -= 1
#         return value

#     wrapper.calls = 0
#     return wrapper



# еще один вариант: как #1, только сбрасывает счетчик
def calls_counter(func):
    first = True
    
    def wrap(*args, **kwargs):
        nonlocal first
        if first:
            first = False
            wrap.calls = 1
            value = func(*args, **kwargs)
            first = True
            return value
        else:
            wrap.calls += 1
            return func(*args, **kwargs)

    wrap.calls = 0
    return wrap


In [11]:
@calls_counter
def simple_recursive(n):
    if n > 0:
        simple_recursive(n - 1)

simple_recursive(3)

assert simple_recursive.calls == 4
print(f'Calls: {simple_recursive.calls}')

Calls: 4


Бывает полезно оптимизировать вызовы "тяжёлых" функций с помощью кеширования.

Кеширование (мемоизация)– это сохранение результатов выполнения функций для предотвращения повторных вычислений.
Перед вызовом функции проверяется есть ли уже вычисленный результат. Если есть – функция не вызывается,
а возвращается сохранённое значение.

Реализуйте декоратор для Least Recently Used (LRU) Cache. Пользователь указывает размер кеша
`N`, и в кеше сохраняются значения для `N` наборов входных параметров функции, т.е. dict пар "входные параметры - результат", причем если в кэше закончилось место, то вытесняется из кеша сначала то,
что использовалось давней всего.

Для решения задачи рекомендую использовать `OrderedDict` в качестве кэша.

Декоратор назовите `@cache`, он должен принимать один параметр – размер кеша. 

Декоратор не должен затирать основные атрибуты функции: `__name__`, `__doc__`, `__module__`. Вам понадобится одна строчка дополнительная строчка для этого (см. ноутбук по теме)

Естественно, вам нельзя пользоваться дефолтным `functools.lru_cache`



In [12]:
from collections import OrderedDict

class LastAccessedOrderedDict(OrderedDict):
    def __getitem__(self, key):
        value = OrderedDict.__getitem__(self, key)
        del self[key]
        self[key] = value
        return value

def cache(cache_size):
    def wrap(func):
        lru_cache = LastAccessedOrderedDict()
        @wraps(func)
        def wrapper(*args, **kwargs):
            key = tuple(args), frozenset(kwargs.items())
            if key not in lru_cache:
                value = func(*args, **kwargs)
                if len(lru_cache) >= cache_size:
                    lru_cache.popitem(last=False)
                lru_cache[key] = value
            return lru_cache[key]
        return wrapper
    return wrap

In [13]:
@cache(2)
def fibo(n):
    fibo.calls += 1
    if n <= 1:
        return n
    return fibo(n - 1) + fibo(n - 2)

fibo.calls = 0
result3 = fibo(3)

assert result3 == 2
assert fibo.calls == 5

fibo.calls = 0
result3 = fibo(3)
assert result3 == 2
assert fibo.calls == 0

fibo(4)
fibo(5)

fibo.calls = 0
result3 = fibo(3)
assert result3 == 2
assert fibo.calls == 0